In [39]:
from langchain_community.llms import Ollama

llm =Ollama(model="gemma2:27b")
# chain 실행
response = llm.invoke("지구의 자전 주기는?")

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
input="나 요즘 나스닥 지수 어떤지 궁금한데 어떻냐"
prompt = ChatPromptTemplate.from_template("비서 llm에게 이런 자료와 명령이 주어졌어. 일단 어떤 행동을 해야할까? 명령:{input} 1. 검색, 2. 요약")
output_parser=StrOutputParser()

chain = prompt|llm|output_parser

chain.invoke({"input":input})

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from langchain.llms import OpenAI
from enum import Enum

요약문=""
행동로그=""
class Action(Enum):
    SEARCH = "검색"
    IDEATION = "아이디에이션"
    SUMMARIZE = "요약"
    #PERSONAL= "개인정보"
    

class ActionOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Action:
        text = text.strip().lower()
        if "검색" in text:
            return Action.SEARCH
        elif "요약" in text:
            return Action.SUMMARIZE
        elif "아이디에이션" in text:
            return Action.IDEATION
        else:
            raise ValueError(f"Invalid action: {text}")

# LLM 초기화
llm =Ollama(model="gemma2:27b")
# 프롬프트 템플릿 생성
분기_prompt = ChatPromptTemplate.from_template(
    "비서 AI에게 다음과 같은 명령이 주어졌습니다: '{input}'\n"
    "이 명령에 대해 어떤 행동을 해야 할까요?\n"
    "1. 검색: 최신 정보나 데이터를 찾아야 하는 경우\n"
    "2. 요약: 기존 정보를 간단히 정리해야 하는 경우\n"
    "3. 아이디에이션: 질문에 대해 총체적 아이디어 및 설계 방향이 필요한 경우\n"
    "위의 것들 중 하나만 선택하여 '검색' 또는 '요약', '아이디에이션' 으로 답변해주세요."
)
통솔_prompt = ChatPromptTemplate.from_template(
    "비서 AI가 다음과 같은 행동을 수행했습니다.: '{요약문} {답변}'\n"
    "여기에서 얻은 정보가 맥락상 유용할 경우 유용한 정보를 추출해서 답변해주세요 "
)
검색분기_prompt = ChatPromptTemplate.from_template(
    "비서 AI가 다음과 같은 행동을 수행했습니다.: '{요약문} {답변}'\n"
    "검색 결과를 요약해주세요."
)
검색요약_prompt = ChatPromptTemplate.from_template(
    "비서 AI가 다음과 같은 행동을 수행했습니다.: '{요약문} {답변}'\n"
    "검색 결과를 요약해주세요."
)
아이디어_prompt = ChatPromptTemplate.from_template(
    "'{요약문}'\n"
    "현재 상황에 기반하여 유용한 솔루션들을 생각해주세요."
)

# 사용자 정의 출력 파서 생성
output_parser = ActionOutputParser()

# 체인 구성
분기_chain = 분기_prompt | llm | output_parser
통솔_chain = 통솔_prompt | llm 

# 체인 실행
input_text = "나 요즘 나스닥 지수 어떤지 궁금한데 어떻냐"
result = chain.invoke({"input": input_text})

print(f"입력: {input_text}")
print(f"결정된 행동: {result.value}")

# 결과에 따른 후속 처리
if result == Action.SEARCH:
    print("검색 작업을 수행합니다...")
    # 여기에 검색 로직을 구현
elif result == Action.SUMMARIZE:
    print("요약 작업을 수행합니다...")
    # 여기에 요약 로직을 구현

입력: 나 요즘 나스닥 지수 어떤지 궁금한데 어떻냐
결정된 행동: 검색
검색 작업을 수행합니다...


In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
def google_search(query):
    driver = webdriver.Chrome()

    try:
        driver.get("https://www.google.com")

        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(query)
        time.sleep(1)  # 검
        search_box.send_keys(Keys.RETURN)
        time.sleep(1)  # 검
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "search"))
        )

        search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        results = []

        for result in search_results[:3]:
            result_dict = {}

            # 제목
            try:
                title_element = result.find_element(By.CSS_SELECTOR, "h3")
                result_dict['title'] = title_element.text
            except:
                result_dict['title'] = "제목 없음"

            # 링크
            try:
                link_element = result.find_element(By.CSS_SELECTOR, "a")
                result_dict['link'] = link_element.get_attribute("href")
            except:
                result_dict['link'] = "링크 없음"


            # 스니펫 (요약 설명)
            try:
                snippet = result.find_element(By.CSS_SELECTOR, "div.VwiC3b").text
                result_dict['snippet'] = snippet
            except:
                result_dict['snippet'] = "스니펫 없음"

            results.append(result_dict)

        return results

    finally:
        driver.quit()

def 본문_추출(link):
    driver = webdriver.Chrome()
    # 본문 내용 추출
    try:
        # 새 탭에서 링크 열기
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(link)

        # 페이지 로딩 대기
        time.sleep(3)  # 페이지 로딩을 위한 대기 시간

        # 본문 추출 (이 부분은 웹사이트마다 다를 수 있음)
        body_element = driver.find_element(By.TAG_NAME, "body")
        내용 = body_element.text[:500]  # 처음 500자만 저장

        # 탭 닫기
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        내용 = "본문 추출 실패"


# for idx, result in enumerate(search_results, 1):
#     결과문 += "{idx}. "+result['title']+"\n   링크:"+result['link']+"\n   설명:"+result['snippet']+"\n\n"
# for idx, result in enumerate(search_results, 1):
#     print(f"{idx}. {result['title']}")
#     print(f"   링크: {result['link']}")
#     print(f"   설명: {result['snippet']}")
#     print()
    

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from typing import List, Dict
요약문="질문:나 요즘 나스닥 지수 어떤지 궁금한데 어떻냐"
# LLM 초기화
llm =Ollama(model="gemma2:27b")
# 1. 검색어 생성 체인
search_query_prompt = ChatPromptTemplate.from_template(
    "다음 정보를 보고 구글에 검색해 정보를 찾기 위한 적절한 검색어를 생성해주세요: {요약문}"
    "검색어는 간단하게 4단어 이내로 설정해주세요"
)
search_query_chain = search_query_prompt | llm | StrOutputParser()

# 2. 관련 글 선택 체인
article_selection_prompt = ChatPromptTemplate.from_template(
    "다음은 '{요약문}'에 대한 검색 결과입니다. 가장 관련성 높고 유용한 선택하세요:\n"
    "{search_results}\n"
    "선택한 글의 번호만 답변해주세요."
)
article_selection_chain = article_selection_prompt | llm | StrOutputParser()

# 3. 최종 요약 생성 체인
final_summary_prompt = ChatPromptTemplate.from_template(
    "다음은 '{요약문}'에 대한 추가 정보 탐색 글입니다. 이 요약문에 주어진 상황을 바탕으로 "
    "질문에 대한 유용하고 간결한 최종 요약을 100자 이내로 작성해주세요.\n"
    "글 내용: {content}\n"
)
final_summary_chain = final_summary_prompt | llm | StrOutputParser()

# 가상의 검색 함수 (실제 구현 필요)
def search_web(query: str) -> List[Dict[str, str]]:
    # 실제로는 여기에 웹 검색 API를 사용하여 결과를 반환해야 합니다.
    return [
        {"title": "나스닥 지수 최근 동향", "link": "http://example.com/1", "snippet": "나스닥 지수가 최근 상승세를 보이고 있습니다..."},
        {"title": "테크 주식 시장 분석", "link": "http://example.com/2", "snippet": "대형 기술주들의 실적 호조로 나스닥 지수가..."},
        {"title": "글로벌 경제 전망", "link": "http://example.com/3", "snippet": "미국 경제 지표 개선으로 나스닥을 비롯한 주요 지수들이..."}
    ]

# 전체 프로세스를 실행하는 함수
def process_query(요약문: str):
    # 1. 검색어 생성
    search_query = search_query_chain.invoke({"요약문": 요약문})
    print(f"생성된 검색어: {search_query}")

    # 2. 웹 검색 (가상)
    search_results = search_web(search_query)


    search_results = google_search(search_query)
    결과문=""
    for idx, result in enumerate(search_results, 1):
        결과문 += str(idx)+ +result['title']+"\n   설명:"+result['snippet']+"\n\n"
    # for idx, result in enumerate(search_results, 1):
    #     print(f"{idx}. {result['title']}")
    #     print(f"   링크: {result['link']}")
    #     print(f"   설명: {result['snippet']}")
    #     print()
        
    # 3. 관련 글 선택
    selected_index = int(article_selection_chain.invoke({
        "요약문": 요약문,
        "search_results": 결과문
    })) - 1
    selected_article = search_results[selected_index]['link']
    본문=본문_추출(selected_article['link'])
    # 4. 최종 요약 생성 (여기서는 snippet을 content로 사용)
    final_summary = final_summary_chain.invoke({
        "content": 본문,
    })

    return final_summary


result = process_query(요약문)
print(f"최종 요약:\n{result}")

생성된 검색어: 나스닥 지수 현황 



StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF69664E1F5+2972373]
	(No symbol) [0x00007FF6962E55F0]
	(No symbol) [0x00007FF6961857FA]
	(No symbol) [0x00007FF696197CDB]
	(No symbol) [0x00007FF69618CF04]
	(No symbol) [0x00007FF69618AE09]
	(No symbol) [0x00007FF69618E5E7]
	(No symbol) [0x00007FF69618E6A0]
	(No symbol) [0x00007FF6961D692C]
	(No symbol) [0x00007FF6961D69F4]
	(No symbol) [0x00007FF6961CC828]
	(No symbol) [0x00007FF6961FBA7A]
	(No symbol) [0x00007FF6961C9266]
	(No symbol) [0x00007FF6961FBC90]
	(No symbol) [0x00007FF69621B8CC]
	(No symbol) [0x00007FF6961FB823]
	(No symbol) [0x00007FF6961C75E8]
	(No symbol) [0x00007FF6961C8751]
	GetHandleVerifier [0x00007FF6966747BD+3129501]
	GetHandleVerifier [0x00007FF69

In [43]:
query = "파이썬 프로그래밍"

search_results = google_search(query)

In [35]:
search_results[2]['link']

'https://codingalzi.github.io/pybook/'

In [36]:
print(결과문)

{idx}. 1. 파이썬 시작하기 - 왕초보를 위한 Python
   링크:https://wikidocs.net/43
   설명:이 강좌는 프로그래밍을 처음 하는 분들을 위해서 파이썬을 통해 프로그래밍의 기초를 차근차근 익힐 수 있도록 진행하려고 합니다. 우리 함께 파이썬 탐험을 떠나 볼까요~

{idx}. 1 장 파이썬(Python) 입문 | 파이썬 프로그래밍 기초
   링크:http://bigdata.dongguk.ac.kr/lectures/Python/_book/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9E%85%EB%AC%B8.html
   설명:1 장 파이썬(Python) 입문 · 프로그램의 생산성과 코드의 가독성, 통계처리 기능 강화 · 효율적인 데이터 구조 · 간단하지만 효과적인 객체지향프로그래밍 · 자유, 오픈소스 ...

{idx}. 파이썬 프로그래밍
   링크:https://codingalzi.github.io/pybook/
   설명:파이썬은 복잡하고 방대한 과제를 해결할 수 있는 능력을 갖추고 있으며 사용법이 C, C++, C#, 자바, 자바스크립트 등 다른 프로그래밍 언어에 비해 상대적으로 쉽다.

{idx}. 
   링크:https://wikidocs.net/7
   설명:스니펫 없음

{idx}. 
   링크:https://www.jetbrains.com/ko-kr/lp/devecosystem-2022/
   설명:스니펫 없음


